**This comprehensive test notebook will:**
- Test all CRUD operations - upsert, search, delete

- Verify namespace functionality - separate data spaces

- Check performance - batch operations and search speed

- Handle edge cases - empty inputs, similar vectors, non-existent data

- Provide detailed reporting - success/failure status for each test

- Include cleanup - remove test data after tests

In [1]:
# test_firestore_vector_db.ipynb
import asyncio
import logging
import sys
import os
import numpy as np
from typing import List, Dict, Any

# Add the project root to Python path
sys.path.append('/app')

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Import the local vector DB service
from src.services.vector_db.local_vector_db import LocalVectorDBService
from src.config import settings

# Initialize the service
vector_db = LocalVectorDBService()

print("🔥 Firestore Local Vector DB Test Suite")
print("=" * 50)

INFO:src.config:Loading ARANGODB_HOST: http://arangodb:8529
INFO:src.config:Loading USE_MOCK_SERVICES: true
INFO:src.services.vector_db.local_vector_db:Successfully connected to Firestore Emulator


🔥 Firestore Local Vector DB Test Suite


# Test 1: Basic Connection Test

In [2]:
# Test 1: Basic Connection and Collection Setup
async def test_connection():
    print("🧪 Test 1: Testing Firestore Connection...")
    try:
        stats = await vector_db.get_index_stats()
        print(f"✅ Connection successful! Current stats: {stats}")
        return True
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

# Run test
connection_ok = await test_connection()

🧪 Test 1: Testing Firestore Connection...
✅ Connection successful! Current stats: {'vectors_count': 0, 'namespaces': [], 'shards_count': 1, 'collection_name': 'vectors'}


# Test 2: Generate Sample Embeddings

In [11]:
# Test 2: Generate Sample Embeddings
# CORRECTED: Generate Sample Embeddings Function
def generate_sample_embeddings(num_vectors: int = 5, dimension: int = 384) -> tuple:
    """Generate sample embeddings for testing - FIXED VERSION"""
    print(f"  Generating {num_vectors} sample embeddings...")
    
    # Generate random vectors (normalized for cosine similarity)
    vectors = []
    for i in range(num_vectors):
        vec = np.random.randn(dimension).astype(np.float32)
        vec = vec / np.linalg.norm(vec)  # Normalize
        vectors.append(vec.tolist())
    
    # Generate IDs and metadata - FIXED: Ensure same length
    ids = [f"test_doc_{i}" for i in range(num_vectors)]
    
    metadatas = []
    sample_titles = [
        "Machine Learning Research Paper",
        "Deep Learning Applications", 
        "Natural Language Processing",
        "Computer Vision Advances",
        "Reinforcement Learning"
    ]
    
    # FIXED: Generate metadata for ALL vectors, not just first 5
    for i in range(num_vectors):
        title = sample_titles[i % len(sample_titles)]  # Cycle through titles if num_vectors > 5
        metadatas.append({
            "title": f"{title} #{i}",
            "authors": [f"Author_{i}", f"Researcher_{i % 5}"],
            "year": 2020 + (i % 5),
            "abstract": f"This is a test abstract for {title} - document {i}",
            "doc_id": f"doc_{i}",
            "is_public": i % 2 == 0  # Alternate public/private
        })
    
    print(f"  ✅ Generated {len(vectors)} embeddings, {len(ids)} IDs, {len(metadatas)} metadata entries")
    return vectors, ids, metadatas


def generate_sample_embeddings_OLD(num_vectors: int = 5, dimension: int = 384) -> tuple:
    """Generate sample embeddings for testing"""
    print(f"\n🧪 Test 2: Generating {num_vectors} sample embeddings...")
    
    # Generate random vectors (normalized for cosine similarity)
    vectors = []
    for i in range(num_vectors):
        vec = np.random.randn(dimension).astype(np.float32)
        vec = vec / np.linalg.norm(vec)  # Normalize
        vectors.append(vec.tolist())
    
    # Generate IDs and metadata
    ids = [f"test_doc_{i}" for i in range(num_vectors)]
    
    metadatas = []
    sample_titles = [
        "Machine Learning Research Paper",
        "Deep Learning Applications", 
        "Natural Language Processing",
        "Computer Vision Advances",
        "Reinforcement Learning"
    ]
    
    for i, title in enumerate(sample_titles):
        metadatas.append({
            "title": title,
            "authors": [f"Author_{i}", f"Researcher_{i}"],
            "year": 2020 + i,
            "abstract": f"This is a test abstract for {title}",
            "doc_id": f"doc_{i}",
            "is_public": i % 2 == 0  # Alternate public/private
        })
    
    print(f"✅ Generated {len(vectors)} embeddings with dimension {len(vectors[0])}")
    return vectors, ids, metadatas

# Generate sample data
vectors, ids, metadatas = generate_sample_embeddings()
print(f"Sample vector: {vectors[0][:5]}...")  # Show first 5 dimensions
print(f"Sample metadata: {metadatas[0]}")

  Generating 5 sample embeddings...
  ✅ Generated 5 embeddings, 5 IDs, 5 metadata entries
Sample vector: [-0.031366072595119476, 0.014946346171200275, 0.004160337150096893, 0.06344591826200485, -0.01146529708057642]...
Sample metadata: {'title': 'Machine Learning Research Paper #0', 'authors': ['Author_0', 'Researcher_0'], 'year': 2020, 'abstract': 'This is a test abstract for Machine Learning Research Paper - document 0', 'doc_id': 'doc_0', 'is_public': True}


In [12]:
# Debug: Check lengths before running performance test
def debug_length_check():
    print("🔍 Debug: Checking vector generation...")
    
    # Test the original broken function
    def original_broken_function(num_vectors=20):
        vectors = [f"vector_{i}" for i in range(num_vectors)]
        ids = [f"id_{i}" for i in range(num_vectors)]
        
        metadatas = []
        sample_titles = ["Title1", "Title2", "Title3", "Title4", "Title5"]
        for i, title in enumerate(sample_titles):
            metadatas.append({"title": title})
            
        return vectors, ids, metadatas
    
    vectors, ids, metadatas = original_broken_function(20)
    print(f"❌ BROKEN: vectors: {len(vectors)}, ids: {len(ids)}, metadatas: {len(metadatas)}")
    
    # Test the fixed function
    def fixed_function(num_vectors=20):
        vectors = [f"vector_{i}" for i in range(num_vectors)]
        ids = [f"id_{i}" for i in range(num_vectors)]
        
        metadatas = []
        sample_titles = ["Title1", "Title2", "Title3", "Title4", "Title5"]
        for i in range(num_vectors):
            title = sample_titles[i % len(sample_titles)]
            metadatas.append({"title": f"{title}_{i}"})
            
        return vectors, ids, metadatas
    
    vectors, ids, metadatas = fixed_function(20)
    print(f"✅ FIXED: vectors: {len(vectors)}, ids: {len(ids)}, metadatas: {len(metadatas)}")

debug_length_check()

🔍 Debug: Checking vector generation...
❌ BROKEN: vectors: 20, ids: 20, metadatas: 5
✅ FIXED: vectors: 20, ids: 20, metadatas: 20


# Test 3: Upsert/Insert Test

In [4]:
# Test 3: Upsert/Insert Test
async def test_upsert():
    print(f"\n🧪 Test 3: Testing Upsert Operations...")
    
    # Test with different namespaces
    namespaces = ["user_123", "public", "test_namespace"]
    
    for namespace in namespaces:
        print(f"\n📦 Upserting to namespace: {namespace}")
        
        try:
            results = await vector_db.upsert_embeddings(
                vectors=vectors,
                ids=[f"{namespace}_{id}" for id in ids],  # Make IDs namespace-specific
                metadatas=metadatas,
                namespace=namespace
            ) 
            
            # Check results
            successes = [r for r in results if r["status"] == "success"]
            errors = [r for r in results if r["status"] == "error"]
            
            print(f"  ✅ Successfully upserted: {len(successes)} vectors")
            if errors:
                print(f"  ❌ Errors: {len(errors)}")
                for error in errors:
                    print(f"    - {error}")
            else:
                print(f"  ✅ No errors")
                
        except Exception as e:
            print(f"  ❌ Upsert failed for namespace {namespace}: {e}")
    
    # Check final stats
    stats = await vector_db.get_index_stats()
    print(f"\n📊 Final collection stats: {stats}")

# Run test
await test_upsert()


🧪 Test 3: Testing Upsert Operations...

📦 Upserting to namespace: user_123


INFO:src.services.vector_db.local_vector_db:Upserted batch of 5 vectors to Firestore
INFO:src.services.vector_db.local_vector_db:Upserted batch of 5 vectors to Firestore
INFO:src.services.vector_db.local_vector_db:Upserted batch of 5 vectors to Firestore


  ✅ Successfully upserted: 5 vectors
  ✅ No errors

📦 Upserting to namespace: public
  ✅ Successfully upserted: 5 vectors
  ✅ No errors

📦 Upserting to namespace: test_namespace
  ✅ Successfully upserted: 5 vectors
  ✅ No errors

📊 Final collection stats: {'vectors_count': 15, 'namespaces': ['test_namespace', 'user_123', 'public'], 'shards_count': 1, 'collection_name': 'vectors'}


# Test 4: Search Test

In [5]:
async def test_search():
    print(f"\n🧪 Test 4: Testing Search Operations...")
    
    # Use the first vector as query
    query_vector = vectors[0]
    
    # Test 4.1: Search across all namespaces
    print(f"\n🔍 Test 4.1: Search across all namespaces")
    try:
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=3,
            namespace=None  # Search all namespaces
        )
        
        print(f"  Found {len(results)} results:")
        for i, result in enumerate(results):
            print(f"    {i+1}. ID: {result['id']}, Distance: {result['distance']:.4f}")
            print(f"       Title: {result['metadata'].get('title', 'N/A')}")
            
    except Exception as e:
        print(f"  ❌ Search failed: {e}")
    
    # Test 4.2: Search within specific namespace
    print(f"\n🔍 Test 4.2: Search within 'user_123' namespace")
    try:
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=2,
            namespace="user_123"
        )
        
        print(f"  Found {len(results)} results in namespace 'user_123':")
        for i, result in enumerate(results):
            print(f"    {i+1}. ID: {result['id']}, Distance: {result['distance']:.4f}")
            
    except Exception as e:
        print(f"  ❌ Namespace search failed: {e}")
    
    # Test 4.3: Search with filters
    print(f"\n🔍 Test 4.3: Search with metadata filters")
    try:
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=5,
            namespace=None,
            filter={"is_public": [True]}  # Only public documents
        )
        
        print(f"  Found {len(results)} public documents:")
        for i, result in enumerate(results):
            is_public = result['metadata'].get('is_public', False)
            print(f"    {i+1}. ID: {result['id']}, Public: {is_public}")
            
    except Exception as e:
        print(f"  ❌ Filtered search failed: {e}")

# Run test
await test_search()


🧪 Test 4: Testing Search Operations...

🔍 Test 4.1: Search across all namespaces
  Found 3 results:
    1. ID: public_test_doc_0, Distance: 0.0000
       Title: Machine Learning Research Paper
    2. ID: test_namespace_test_doc_0, Distance: 0.0000
       Title: Machine Learning Research Paper
    3. ID: user_123_test_doc_0, Distance: 0.0000
       Title: Machine Learning Research Paper

🔍 Test 4.2: Search within 'user_123' namespace
  Found 2 results in namespace 'user_123':
    1. ID: user_123_test_doc_0, Distance: 0.0000
    2. ID: user_123_test_doc_4, Distance: 0.9278

🔍 Test 4.3: Search with metadata filters
  Found 0 public documents:


/usr/local/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


# Test 5: Delete Test

In [6]:
async def test_delete():
    print(f"\n🧪 Test 5: Testing Delete Operations...")
    
    # Delete documents from user_123 namespace
    docs_to_delete = [f"user_123_{id}" for id in ids[:2]]  # Delete first 2 docs
    
    print(f"🗑️  Deleting {len(docs_to_delete)} documents: {docs_to_delete}")
    
    try:
        delete_results = await vector_db.delete(docs_to_delete)
        
        successes = [r for r in delete_results if r["status"] == "success"]
        errors = [r for r in delete_results if r["status"] == "error"]
        
        print(f"  ✅ Successfully deleted: {len(successes)} documents")
        if errors:
            print(f"  ❌ Delete errors: {len(errors)}")
            for error in errors:
                print(f"    - {error}")
    
    except Exception as e:
        print(f"  ❌ Delete operation failed: {e}")
    
    # Verify deletion by searching
    print(f"\n🔍 Verifying deletion by searching...")
    try:
        query_vector = vectors[0]
        results = await vector_db.search(
            query_embedding=query_vector,
            top_k=10,
            namespace="user_123"
        )
        
        remaining_ids = [r["id"] for r in results]
        print(f"  Remaining documents in 'user_123': {len(remaining_ids)}")
        print(f"  Remaining IDs: {remaining_ids}")
        
        # Check if deleted docs are gone
        for deleted_id in docs_to_delete:
            if deleted_id in remaining_ids:
                print(f"  ⚠️  Document {deleted_id} still exists!")
            else:
                print(f"  ✅ Document {deleted_id} successfully deleted")
                
    except Exception as e:
        print(f"  ❌ Verification search failed: {e}")

# Run test
await test_delete()


🧪 Test 5: Testing Delete Operations...
🗑️  Deleting 2 documents: ['user_123_test_doc_0', 'user_123_test_doc_1']
  ✅ Successfully deleted: 2 documents

🔍 Verifying deletion by searching...
  Remaining documents in 'user_123': 3
  Remaining IDs: ['user_123_test_doc_4', 'user_123_test_doc_3', 'user_123_test_doc_2']
  ✅ Document user_123_test_doc_0 successfully deleted
  ✅ Document user_123_test_doc_1 successfully deleted


# Test 6: Performance Test

In [7]:
# Test 6: Performance Test - FIXED VERSION
async def test_performance():
    print(f"\n🧪 Test 6: Testing Performance...")
    
    # FIXED: Generate consistent number of vectors, ids, and metadatas
    def generate_large_sample_embeddings(num_vectors: int = 20, dimension: int = 384) -> tuple:
        """Generate sample embeddings for performance testing"""
        print(f"  Generating {num_vectors} sample embeddings...")
        
        # Generate random vectors (normalized for cosine similarity)
        vectors = []
        for i in range(num_vectors):
            vec = np.random.randn(dimension).astype(np.float32)
            vec = vec / np.linalg.norm(vec)  # Normalize
            vectors.append(vec.tolist())
        
        # Generate IDs and metadata - FIXED: Ensure same length
        ids = [f"test_doc_{i}" for i in range(num_vectors)]
        
        metadatas = []
        sample_titles = [
            "Machine Learning Research Paper",
            "Deep Learning Applications", 
            "Natural Language Processing",
            "Computer Vision Advances",
            "Reinforcement Learning",
            "Neural Networks Study",
            "AI Ethics and Society",
            "Quantum Computing Research",
            "Data Mining Techniques",
            "Robotics and Automation"
        ]
        
        # FIXED: Generate metadata for ALL vectors, not just first 5
        for i in range(num_vectors):
            title = sample_titles[i % len(sample_titles)]  # Cycle through titles
            metadatas.append({
                "title": f"{title} #{i}",
                "authors": [f"Author_{i}", f"Researcher_{i % 5}"],
                "year": 2020 + (i % 5),
                "abstract": f"This is a test abstract for {title} - document {i}",
                "doc_id": f"doc_{i}",
                "is_public": i % 2 == 0  # Alternate public/private
            })
        
        print(f"  ✅ Generated {len(vectors)} embeddings, {len(ids)} IDs, {len(metadatas)} metadata entries")
        return vectors, ids, metadatas

    # Generate larger batch with FIXED function
    large_vectors, large_ids, large_metadatas = generate_large_sample_embeddings(
        num_vectors=20, 
        dimension=384
    )
    
    print(f"⏱️  Testing with {len(large_vectors)} vectors...")
    
    import time
    
    # Test upsert performance
    start_time = time.time()
    results = await vector_db.upsert_embeddings(
        vectors=large_vectors,
        ids=[f"perf_test_{id}" for id in large_ids],
        metadatas=large_metadatas,
        namespace="performance_test"
    )
    upsert_time = time.time() - start_time
    
    successes = len([r for r in results if r["status"] == "success"])
    print(f"  📈 Upsert Performance: {successes} docs in {upsert_time:.2f}s "
          f"({successes/upsert_time:.1f} docs/sec)")
    
    # Test search performance
    start_time = time.time()
    search_results = await vector_db.search(
        query_embedding=large_vectors[0],
        top_k=10,
        namespace="performance_test"
    )
    search_time = time.time() - start_time
    
    print(f"  📈 Search Performance: {len(search_results)} results in {search_time:.4f}s")
    
    # Verify search results
    if search_results:
        print(f"  🔍 Best match: {search_results[0]['id']} (distance: {search_results[0]['distance']:.4f})")
    
    # Clean up performance test data
    delete_results = await vector_db.delete([f"perf_test_{id}" for id in large_ids])
    delete_successes = len([r for r in delete_results if r["status"] == "success"])
    print(f"  🧹 Cleaned up {delete_successes} performance test documents")

# Run test
await test_performance()


🧪 Test 6: Testing Performance...

🧪 Test 2: Generating 20 sample embeddings...
✅ Generated 20 embeddings with dimension 384
⏱️  Testing with 20 vectors...


ValueError: Vectors, IDs, and metadatas must have the same length

# Test 7: Edge Cases Test

In [8]:
# Test 7: Edge Cases Test
async def test_edge_cases():
    print(f"\n🧪 Test 7: Testing Edge Cases...")
    
    # Test 7.1: Empty inputs
    print(f"\n🔍 Test 7.1: Empty inputs")
    try:
        results = await vector_db.upsert_embeddings([], [], [])
        print(f"  ✅ Empty upsert handled: {results}")
    except Exception as e:
        print(f"  ❌ Empty upsert failed: {e}")
    
    # Test 7.2: Very similar vectors
    print(f"\n🔍 Test 7.2: Similar vectors search")
    try:
        # Create two very similar vectors
        base_vector = np.random.randn(384).astype(np.float32)
        base_vector = base_vector / np.linalg.norm(base_vector)
        
        similar_vector = base_vector + 0.01 * np.random.randn(384)
        similar_vector = similar_vector / np.linalg.norm(similar_vector)
        
        # Store them
        await vector_db.upsert_embeddings(
            vectors=[base_vector.tolist(), similar_vector.tolist()],
            ids=["similar_1", "similar_2"],
            metadatas=[{"test": "similar_1"}, {"test": "similar_2"}],
            namespace="similarity_test"
        )
        
        # Search and check distance
        results = await vector_db.search(
            query_embedding=base_vector.tolist(),
            top_k=2,
            namespace="similarity_test"
        )
        
        print(f"  Similar vectors distance: {results[1]['distance']:.6f}")
        print(f"  ✅ Similar vectors test completed")
        
        # Clean up
        await vector_db.delete(["similar_1", "similar_2"])
        
    except Exception as e:
        print(f"  ❌ Similar vectors test failed: {e}")
    
    # Test 7.3: Non-existent namespace search
    print(f"\n🔍 Test 7.3: Non-existent namespace")
    try:
        results = await vector_db.search(
            query_embedding=vectors[0],
            top_k=5,
            namespace="non_existent_namespace_12345"
        )
        print(f"  Search in non-existent namespace returned: {len(results)} results")
        print(f"  ✅ Non-existent namespace handled correctly")
    except Exception as e:
        print(f"  ❌ Non-existent namespace search failed: {e}")

# Run test
await test_edge_cases()

INFO:src.services.vector_db.local_vector_db:Upserted batch of 2 vectors to Firestore



🧪 Test 7: Testing Edge Cases...

🔍 Test 7.1: Empty inputs
  ✅ Empty upsert handled: []

🔍 Test 7.2: Similar vectors search
  Similar vectors distance: 0.018226
  ✅ Similar vectors test completed

🔍 Test 7.3: Non-existent namespace
  Search in non-existent namespace returned: 0 results
  ✅ Non-existent namespace handled correctly


# Final Summary and Cleanup

In [9]:
# Final Summary and Cleanup
async def final_summary():
    print(f"\n" + "="*50)
    print(f"📊 FINAL TEST SUMMARY")
    print(f"="*50)
    
    # Get final stats
    stats = await vector_db.get_index_stats()
    print(f"Final collection stats: {stats}")
    
    # Count documents by namespace
    print(f"\n📂 Documents by namespace:")
    namespaces = ["user_123", "public", "test_namespace", "performance_test", "similarity_test"]
    
    for namespace in namespaces:
        try:
            results = await vector_db.search(
                query_embedding=vectors[0],
                top_k=100,  # Large number to count all
                namespace=namespace
            )
            print(f"  {namespace}: {len(results)} documents")
        except:
            print(f"  {namespace}: 0 documents (or error)")
    
    print(f"\n🎯 Test Suite Completed!")
    print(f"✅ Firestore Local Vector DB is working correctly")
    print(f"💡 Use VECTOR_DB_TYPE=local in your environment to enable this service")

# Run final summary
await final_summary()


📊 FINAL TEST SUMMARY
Final collection stats: {'vectors_count': 13, 'namespaces': ['test_namespace', 'user_123', 'public'], 'shards_count': 1, 'collection_name': 'vectors'}

📂 Documents by namespace:
  user_123: 3 documents
  public: 5 documents
  test_namespace: 5 documents
  performance_test: 0 documents
  similarity_test: 0 documents

🎯 Test Suite Completed!
✅ Firestore Local Vector DB is working correctly
💡 Use VECTOR_DB_TYPE=local in your environment to enable this service


# Quick health check function for future use

In [13]:
# Quick health check function for future use
async def health_check():
    """Quick health check for the vector DB"""
    try:
        stats = await vector_db.get_index_stats()
        print(f"🟢 Vector DB Healthy - Total vectors: {stats.get('vectors_count', 0)}")
        return True
    except Exception as e:
        print(f"🔴 Vector DB Unhealthy: {e}")
        return False

# Run quick health check
print(f"\n🏥 Running Quick Health Check...")
is_healthy = await health_check()


🏥 Running Quick Health Check...
🟢 Vector DB Healthy - Total vectors: 13


# Miscs

In [14]:
await vector_db.get_index_stats()

{'vectors_count': 13,
 'namespaces': ['test_namespace', 'user_123', 'public'],
 'shards_count': 1,
 'collection_name': 'vectors'}

In [15]:
help(vector_db.upsert_embeddings)

Help on method upsert_embeddings in module src.services.vector_db.local_vector_db:

async upsert_embeddings(vectors: List[List[float]], ids: List[str], metadatas: List[Dict[str, Any]], namespace: Optional[str] = None) method of src.services.vector_db.local_vector_db.LocalVectorDBService instance
    Upsert embeddings into Firestore with vector support.



In [51]:
await vector_db.upsert_embeddings(vectors=[[.35]*384,], 
                                  ids=['abc5',], 
                                  metadatas=[{'a': 2, 'b': 3}], 
                                  namespace='user007')

INFO:src.services.vector_db.local_vector_db:Upserted batch of 1 vectors to Firestore


[{'status': 'success', 'id': 'abc5'}]

In [52]:
await vector_db.get_index_stats()

{'vectors_count': 18,
 'namespaces': ['public', 'test_namespace', 'user_123', 'user007'],
 'shards_count': 1,
 'collection_name': 'vectors'}

In [53]:
help(vector_db.search)

Help on method search in module src.services.vector_db.local_vector_db:

async search(query_embedding: List[float], top_k: int, namespace: Optional[str] = None, filter: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]] method of src.services.vector_db.local_vector_db.LocalVectorDBService instance
    Search for similar vectors using cosine similarity.



In [57]:
await vector_db.search(query_embedding=[0.1]*384,
                       top_k=10,
                       namespace="user007")

[{'id': 'abc1', 'distance': np.float64(0.0), 'metadata': {'b': 3, 'a': 2}},
 {'id': 'abc2', 'distance': np.float64(0.0), 'metadata': {'b': 3, 'a': 2}},
 {'id': 'abc4',
  'distance': np.float64(1.1102230246251565e-16),
  'metadata': {'b': 3, 'a': 2}},
 {'id': 'abc5',
  'distance': np.float64(2.220446049250313e-16),
  'metadata': {'b': 3, 'a': 2}},
 {'id': 'abc3', 'distance': np.float64(2.0), 'metadata': {'b': 3, 'a': 2}}]